In [1]:
pip install polars

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Notebook to run after q Tues run of after run of cms_api.py outputs upated nh_pre_proc_raw.csv file

# import libraries
import polars as pl
import pandas as pd
import pyarrow.parquet as pq
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
#import plotly.express as px
#import plotly.graph_objs as go
#import geopandas as gpd
# import osmnx as ox
import folium
import geopandas as gpd
# import geopy
# from geopy.geocoders import Nominatim
import json
import re

from urllib.request import urlopen


from datetime import datetime, timedelta, date
import os
import time
import sys
import requests
# import sidetable
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [4]:
m = folium.Map()
m.save("footprint.html")

In [5]:
# #read FIPS dataset
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
#     counties = json.load(response)

In [6]:
#read FIPS dataset
fips_df_raw = pd.read_csv("data_pre_proc/fips2county.tsv", sep='\t', header='infer', dtype=str, encoding='latin-1')

In [7]:
fips_df_raw.head(10)

,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,CountyCBSA
0,01,001,Autauga,Alabama,01001,AL,AL | AUTAUGA,33860
1,01,003,Baldwin,Alabama,01003,AL,AL | BALDWIN,19300
2,01,005,Barbour,Alabama,01005,AL,AL | BARBOUR,21640
3,01,007,Bibb,Alabama,01007,AL,AL | BIBB,13820
4,01,009,Blount,Alabama,01009,AL,AL | BLOUNT,13820
5,01,011,Bullock,Alabama,01011,AL,AL | BULLOCK,NaN
6,01,013,Butler,Alabama,01013,AL,AL | BUTLER,NaN
7,01,015,Calhoun,Alabama,01015,AL,AL | CALHOUN,11500
8,01,017,Chambers,Alabama,01017,AL,AL | CHAMBERS,29300
9,01,019,Cherokee,Alabama,01019,AL,AL | CHEROKEE,NaN


In [8]:
# fips_df.loc[fips_df['StateAbbr']=='LA']

In [9]:
#drop unneeded columns
fips_df = fips_df_raw.drop(columns=["StateFIPS",
                                "CountyFIPS_3",
                                "STATE_COUNTY",
                                "CountyCBSA"])

In [10]:
# geolocator = Nominatim(user_agent="SW's mapping app", timeout=2)


In [11]:
#read in CMS Nursing Home Covid-19 data
nh_pre_proc_df_raw = pd.read_csv("data_pre_proc/nh_pre_proc_raw.csv")
nh_pre_proc_df_raw.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_total_confirmed_covid_19,residents_weekly_all_deaths,residents_total_all_deaths,residents_weekly_covid_19_deaths,residents_total_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,staff_total_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,total_resident_confirmed_covid_19_cases_per_1_000_residents,total_resident_covid_19_deaths_per_1_000_residents,Number_of_Residents_who_are_Up_to_Date_on_COVID_19_Vaccinations__14_Days_or_More_Before_Positive_Test,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines
0,2023-10-29,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,75.0,0.0,54.0,0.0,5.0,57.0,51.0,0.0,0.0,0.0,66.0,0.0,0.0,1470.59,98.04,0.0,51.0,81.0,70.0,NaN,86.420,47.0,NaN,92.157,24.0,NaN,29.630
1,2023-10-29,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,86.0,0.0,5.0,0.0,2.0,85.0,80.0,0.0,0.0,0.0,152.0,0.0,0.0,1075.00,25.00,0.0,80.0,185.0,142.0,NaN,76.757,48.0,NaN,60.000,1.0,NaN,0.541
2,2023-10-29,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,23.0,0.0,44.0,0.0,3.0,50.0,39.0,0.0,0.0,0.0,82.0,0.0,0.0,589.74,76.92,0.0,38.0,79.0,64.0,NaN,81.013,0.0,NaN,0.000,0.0,NaN,0.000
3,2023-10-29,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,108.0,0.0,33.0,0.0,2.0,90.0,76.0,0.0,0.0,0.0,126.0,0.0,0.0,1421.05,26.32,0.0,77.0,101.0,77.0,NaN,76.238,70.0,NaN,90.909,8.0,NaN,7.921
4,2023-10-29,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,94.0,0.0,152.0,0.0,0.0,103.0,81.0,0.0,0.0,0.0,44.0,0.0,0.0,1160.49,0.00,0.0,83.0,155.0,129.0,NaN,83.226,62.0,NaN,74.699,0.0,NaN,0.000
5,2023-10-29,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,0.0,233.0,0.0,93.0,0.0,0.0,149.0,130.0,0.0,0.0,0.0,249.0,0.0,0.0,1792.31,0.00,0.0,144.0,249.0,104.0,NaN,41.767,0.0,NaN,0.000,0.0,NaN,0.000
6,2023-10-29,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,0.0,142.0,1.0,132.0,0.0,7.0,124.0,98.0,0.0,0.0,0.0,179.0,0.0,0.0,1448.98,71.43,0.0,100.0,82.0,77.0,NaN,93.902,0.0,NaN,0.000,5.0,NaN,6.098
7,2023-10-29,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,0.0,180.0,0.0,94.0,0.0,26.0,201.0,89.0,0.0,0.0,0.0,227.0,0.0,0.0,2022.47,292.13,0.0,89.0,139.0,111

In [12]:
# columns in nh_pre_proc_df
list(nh_pre_proc_df_raw.columns.values)

['week_ending',
 'federal_provider_number',
 'provider_name',
 'provider_address',
 'provider_city',
 'provider_state',
 'provider_zip_code',
 'provider_phone_number',
 'county',
 'submitted_data',
 'passed_quality_assurance_check',
 'residents_weekly_confirmed_covid_19',
 'residents_total_confirmed_covid_19',
 'residents_weekly_all_deaths',
 'residents_total_all_deaths',
 'residents_weekly_covid_19_deaths',
 'residents_total_covid_19_deaths',
 'number_of_all_beds',
 'total_number_of_occupied_beds',
 'Residents_Hospitalizations_with_Confirmed_COVID_19',
 'Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines',
 'staff_weekly_confirmed_covid_19',
 'staff_total_confirmed_covid_19',
 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
 'weekly_resident_covid_19_deaths_per_1_000_residents',
 'total_resident_confirmed_covid_19_cases_per_1_000_residents',
 'total_resident_covid_19_deaths_per_1_000_residents',
 'Number_of_Residents_who_are_Up_to_Date_on_C

In [13]:
# drop unneeded rows from nh pre proc df
nh_pre_proc_df = nh_pre_proc_df_raw.drop(columns=["residents_total_confirmed_covid_19",
                                              "residents_total_all_deaths",
                                              "residents_total_covid_19_deaths",
                                              "staff_total_confirmed_covid_19",
                                              "total_resident_confirmed_covid_19_cases_per_1_000_residents",
                                              "total_resident_covid_19_deaths_per_1_000_residents",
                                              "Number_of_Residents_who_are_Up_to_Date_on_COVID_19_Vaccinations__14_Days_or_More_Before_Positive_Test", 
                                                 ])


In [14]:
# Get rows from fips df where state is not in nh pre proc df
fips_state_notin_nh = fips_df[~fips_df['StateAbbr'].isin(nh_pre_proc_df['provider_state'])]
fips_state_notin_nh

,CountyName,StateName,CountyFIPS,StateAbbr


In [15]:
# Get rows where provider_state values from nh_pre_proc_df are not in fips_df
nh_state_notin_fips = nh_pre_proc_df[~nh_pre_proc_df['provider_state'].isin(fips_df['StateAbbr'])]
nh_state_notin_fips.provider_state.unique()

array(['PR', 'GU'], dtype=object)

In [16]:
# Drop rows from nh_pre_proc_df where provider_state values are not in fips_df
nh_pre_proc_df = nh_pre_proc_df[~nh_pre_proc_df['provider_state'].isin(nh_state_notin_fips['provider_state'])]
nh_pre_proc_df

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines
0,2023-10-29,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,51.0,0.0,0.0,0.0,0.0,0.0,51.0,81.0,70.0,NaN,86.420,47.0,NaN,92.157,24.0,NaN,29.630
1,2023-10-29,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,80.0,0.0,0.0,0.0,0.0,0.0,80.0,185.0,142.0,NaN,76.757,48.0,NaN,60.000,1.0,NaN,0.541
2,2023-10-29,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,39.0,0.0,0.0,0.0,0.0,0.0,38.0,79.0,64.0,NaN,81.013,0.0,NaN,0.000,0.0,NaN,0.000
3,2023-10-29,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,0.0,0.0,90.0,76.0,0.0,0.0,0.0,0.0,0.0,77.0,101.0,77.0,NaN,76.238,70.0,NaN,90.909,8.0,NaN,7.921
4,2023-10-29,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,0.0,0.0,103.0,81.0,0.0,0.0,0.0,0.0,0.0,83.0,155.0,129.0,NaN,83.226,62.0,NaN,74.699,0.0,NaN,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104412,2023-12-10,745017,THE LODGE OF SAGINAW HEALTH AND WELLNESS,848 W MCLEROY BLVD,SAGINAW,TX,76179,6859001210,Tarrant,Y,Y,0.0,4.0,0.0,130.0,101.0,0.0,0.0,2.0,0.0,0.0,110.0,144.0,91.0,63.194,63.194,0.0,0.0,0.000,0.0,0.0,0.000
104413,2023-12-10,745019,JAMES L. WEST ALZHEIMER'S CENTER,1111 SUMMIT AVE,FORT WORTH,TX,76102,8178771199,Tarrant,N,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104414,2023-12-10,745021,LINDALE SPECIALTY CARE CENTER,13905 FM 2710,LINDALE,TX,75771,4302602300,Smith,Y,Y,0.0,0.0,0.0,122.0,87.0,0.0,0.0,0.0,0.0,0.0,87.0,140.0,50.0,35.714,35.714,0.0,0.0,0.000,0.0,0.0,0.000
104415,2023-12-10,745022,WARE MEMORIAL CARE CENTER,1510 S. VAN BUREN ST.,AMARILLO,TX,79101,8063730471,Potter,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# # keep only alphanumberic chars in column values
# def alphanum(element):
    
#     return "".join(filter(str.isalnum, element))

In [18]:

# nh_pre_proc_df['county_rev'] = [alphanum(x) for x in nh_pre_proc_df['county']]

In [19]:
# nh_pre_proc_df['county_rev']

In [20]:
# fips_df['CountyName_rev'] = [alphanum(x) for x in fips_df['CountyName']]
# fips_df['CountyName_rev']

In [21]:
fips_df.loc[fips_df['StateAbbr']=='LA','CountyName']


1114                  Acadia Parish
1115                   Allen Parish
1116               Ascension Parish
1117              Assumption Parish
1118               Avoyelles Parish
1119              Beauregard Parish
1120               Bienville Parish
1121                 Bossier Parish
1122                   Caddo Parish
1123               Calcasieu Parish
1124                Caldwell Parish
1125                 Cameron Parish
1126               Catahoula Parish
1127               Claiborne Parish
1128               Concordia Parish
1129                 De Soto Parish
1130        East Baton Rouge Parish
1131            East Carroll Parish
1132          East Feliciana Parish
1133              Evangeline Parish
1134                Franklin Parish
1135                   Grant Parish
1136                  Iberia Parish
1137               Iberville Parish
1138                 Jackson Parish
1139               Jefferson Parish
1140         Jefferson Davis Parish
1141               Lafayette

In [22]:
sorted(list(nh_pre_proc_df_raw.loc[nh_pre_proc_df_raw['provider_state']=='LA', 'county'].unique()),reverse=False)


['Acadia',
 'Allen',
 'Ascension',
 'Assumption',
 'Avoyelles',
 'Beauregard',
 'Bienville',
 'Bossier',
 'Caddo',
 'Calcasieu',
 'Caldwell',
 'Catahoula',
 'Claiborne',
 'Concordia',
 'De Soto',
 'East Baton Rouge',
 'East Carroll',
 'East Feliciana',
 'Evangeline',
 'Franklin',
 'Grant',
 'Iberia',
 'Iberville',
 'Jackson',
 'Jefferson',
 'Jefferson Davis',
 'LaSalle',
 'Lafayette',
 'Lafourche',
 'Lincoln',
 'Livingston',
 'Madison',
 'Morehouse',
 'Natchitoches',
 'Orleans',
 'Ouachita',
 'Plaquemines',
 'Pointe Coupee',
 'Rapides',
 'Red River',
 'Richland',
 'Sabine',
 'St. Charles',
 'St. Helena',
 'St. James',
 'St. John the Baptist',
 'St. Landry',
 'St. Martin',
 'St. Mary',
 'St. Tammany',
 'Tangipahoa',
 'Terrebonne',
 'Union',
 'Vermilion',
 'Vernon',
 'Washington',
 'Webster',
 'West Baton Rouge',
 'West Carroll',
 'West Feliciana',
 'Winn']

In [23]:
# Issue: counties in LA have 'Parish' after them in the FIPS dataset, so we will remove 'Parish' from LA county names'
def remove_parish_la(value):
    return value.rstrip(' Parish')


In [24]:
fips_df['CountyName_rev'] = fips_df.apply(lambda row: remove_parish_la(row['CountyName']) if row['StateAbbr'] == 'LA'
                                               else row['CountyName'], axis=1)

In [25]:
# in case any 'Parish' in nh pre proc df, romove those too
nh_pre_proc_df['county_rev'] = nh_pre_proc_df.apply(lambda row: remove_parish_la(row['county']) if row['provider_state'] == 'LA'
                                               else row['county'], axis=1)



In [26]:
# Issue: AK counties don't align between datasets
sorted(list(nh_pre_proc_df.loc[nh_pre_proc_df['provider_state']=='AK','county'].unique()),reverse=False)


['Anchorage',
 'Bethel',
 'Cordova-Mccarthy',
 'Fairbanks',
 'Juneau',
 'Kenai-Cook Inlet',
 'Ketchikan',
 'Kodiak Island Borough',
 'Matanuska-Susitna',
 'Nome',
 'Not Available',
 'Seward',
 'Sitka',
 'Valdez-Chitina-Whittier',
 'Wrangell-Petersburg']

In [27]:
sorted(list(fips_df.loc[fips_df['StateAbbr']=='AK','CountyName_rev'].unique()),reverse=False)


['Aleutians East Borough',
 'Aleutians West Census Area',
 'Anchorage Municipality',
 'Bethel Census Area',
 'Bristol Bay Borough',
 'Chugach Census Area',
 'Copper River Census Area',
 'Denali Borough',
 'Dillingham Census Area',
 'Fairbanks North Star Borough',
 'Haines Borough',
 'Hoonah-Angoon Census Area',
 'Juneau City and Borough',
 'Kenai Peninsula Borough',
 'Ketchikan Gateway Borough',
 'Kodiak Island Borough',
 'Kusilvak Census Area',
 'Lake and Peninsula Borough',
 'Matanuska-Susitna Borough',
 'Nome Census Area',
 'North Slope Borough',
 'Northwest Arctic Borough',
 'Petersburg Borough',
 'Prince of Wales-Hyder Census Area',
 'Sitka City and Borough',
 'Skagway Municipality',
 'Southeast Fairbanks Census Area',
 'Wrangell City and Borough',
 'Yakutat City and Borough',
 'Yukon-Koyukuk Census Area']

In [28]:
# fips_df['CountyName_rev'] = fips_df.apply(lambda row: remove_ak_counties_ends(row['CountyName_rev']) if row['StateAbbr'] == 'AK'
#                                                else row['CountyName_rev'], axis=1)

In [29]:
fips_df['CountyName_rev']

0          Autauga
1          Baldwin
2          Barbour
3             Bibb
4           Blount
           ...    
3138    Sweetwater
3139         Teton
3140         Uinta
3141      Washakie
3142        Weston
Name: CountyName_rev, Length: 3143, dtype: object

In [30]:
fips_df['CountyName_rev']=np.where(fips_df['StateAbbr']=='AK', fips_df['CountyName_rev'].str.replace(' City and Borough| North Star Borough| Municipality| Census Area| Gateway Borough','',regex=True),fips_df['CountyName_rev'])



In [31]:
# Issue: County name diff in AK
    
fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'Kenai Peninsula Borough': 'Kenai-Cook Inlet'}).where(fips_df['StateAbbr']=='AK', fips_df['CountyName_rev'])


In [32]:
sorted(list(nh_pre_proc_df.loc[nh_pre_proc_df['provider_state']=='AK','county_rev'].unique()),reverse=False)

['Anchorage',
 'Bethel',
 'Cordova-Mccarthy',
 'Fairbanks',
 'Juneau',
 'Kenai-Cook Inlet',
 'Ketchikan',
 'Kodiak Island Borough',
 'Matanuska-Susitna',
 'Nome',
 'Not Available',
 'Seward',
 'Sitka',
 'Valdez-Chitina-Whittier',
 'Wrangell-Petersburg']

In [33]:
sorted(list(fips_df.loc[fips_df['StateAbbr']=='AK','CountyName_rev'].unique()),reverse=False)

['Aleutians East Borough',
 'Aleutians West',
 'Anchorage',
 'Bethel',
 'Bristol Bay Borough',
 'Chugach',
 'Copper River',
 'Denali Borough',
 'Dillingham',
 'Fairbanks',
 'Haines Borough',
 'Hoonah-Angoon',
 'Juneau',
 'Kenai-Cook Inlet',
 'Ketchikan',
 'Kodiak Island Borough',
 'Kusilvak',
 'Lake and Peninsula Borough',
 'Matanuska-Susitna Borough',
 'Nome',
 'North Slope Borough',
 'Northwest Arctic Borough',
 'Petersburg Borough',
 'Prince of Wales-Hyder',
 'Sitka',
 'Skagway',
 'Southeast Fairbanks',
 'Wrangell',
 'Yakutat',
 'Yukon-Koyukuk']

In [34]:
# Issue: differences in VA counties
# Revise CountyName_rev in fips df to match those in nh pre proc df


fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'Albemarle': 'Albermarle',  'Bedford':'Bedford City',
 'Poquoson city':'Poquoson'}).where(fips_df['StateAbbr']=='VA', fips_df['CountyName_rev'])

In [35]:
# Issue: non alpha characters in NM county

fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'DoÃ\x83Â±a Ana': 'Dona Ana'}).where(fips_df['StateAbbr']=='NM', fips_df['CountyName_rev'])


In [36]:
# Issue: Spelling diff in NH county

fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'Hillsborough':'Hillsboro'}).where(fips_df['StateAbbr']=='NH', fips_df['CountyName_rev'])


In [37]:
nh_pre_proc_df['state_county'] = (nh_pre_proc_df['provider_state'].fillna('')
                                  + nh_pre_proc_df['county_rev'].fillna('')).astype(str).str.lower()



fips_df['state_county'] = (fips_df['StateAbbr'].fillna('') + fips_df['CountyName_rev'].fillna('')).astype(str).str.lower()

# nh_pre_proc_df_reset = nh_pre_proc_df.reset_index(drop=True)
# fips_df_reset = fips_df.reset_index(drop=True)

In [38]:
#which are in fips_df_sc but not in nh_pre_proc_df_sc?

not_in_nh=list(set(fips_df['state_county']) - set(nh_pre_proc_df['state_county']))
not_in_nh_sorted = sorted(not_in_nh, reverse=False)
not_in_nh_sorted

['akaleutians east borough',
 'akaleutians west',
 'akbristol bay borough',
 'akchugach',
 'akcopper river',
 'akdenali borough',
 'akdillingham',
 'akhaines borough',
 'akhoonah-angoon',
 'akkusilvak',
 'aklake and peninsula borough',
 'akmatanuska-susitna borough',
 'aknorth slope borough',
 'aknorthwest arctic borough',
 'akpetersburg borough',
 'akprince of wales-hyder',
 'akskagway',
 'aksoutheast fairbanks',
 'akwrangell',
 'akyakutat',
 'akyukon-koyukuk',
 'arcalhoun',
 'ardesha',
 'arnewton',
 'azgreenlee',
 'azla paz',
 'azsanta cruz',
 'caalpine',
 'camono',
 'casierra',
 'coclear creek',
 'cocostilla',
 'cocuster',
 'codolores',
 'coelbert',
 'cogilpin',
 'cogrand',
 'cohinsdale',
 'cojackson',
 'cokiowa',
 'colake',
 'comineral',
 'coouray',
 'copark',
 'copitkin',
 'cosaguache',
 'cosan juan',
 'cosan miguel',
 'cosummit',
 'flglades',
 'flliberty',
 'flunion',
 'gaatkinson',
 'gabaker',
 'gabanks',
 'gachattahoochee',
 'gadawson',
 'gadooly',
 'gaechols',
 'galincoln',
 '

In [39]:
#which are in nh_pre_proc_df_sc but not in fips_df_sc?

not_in_fips=list(set(nh_pre_proc_df['state_county']) - set(fips_df['state_county']))
not_in_fips_sorted = sorted(not_in_fips, reverse=False)
not_in_fips_sorted

# WIP 5 state_counties in AK in fips data that have no direct match in nh pre proc data: 
# 'akcordova-mccarthy',
#  'akmatanuska-susitna',
#  'aknot available',
#  'akseward',
#  'akvaldez-chitina-whittier',
#  'akwrangell-petersburg'

['akcordova-mccarthy',
 'akmatanuska-susitna',
 'aknot available',
 'akseward',
 'akvaldez-chitina-whittier',
 'akwrangell-petersburg']

In [40]:
#merge nh_pre_proc_df and fips df on state_county
nh_pre_proc_fips_merge = nh_pre_proc_df.merge(fips_df, how='left', left_on=["state_county"], right_on=["state_county"])


In [41]:
nh_pre_proc_fips_merge.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,county_rev,state_county,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev
0,2023-10-29,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,51.0,0.0,0.0,0.0,0.0,0.0,51.0,81.0,70.0,NaN,86.420,47.0,NaN,92.157,24.0,NaN,29.630,Franklin,alfranklin,Franklin,Alabama,01059,AL,Franklin
1,2023-10-29,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,80.0,0.0,0.0,0.0,0.0,0.0,80.0,185.0,142.0,NaN,76.757,48.0,NaN,60.000,1.0,NaN,0.541,Talladega,altalladega,Talladega,Alabama,01121,AL,Talladega
2,2023-10-29,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,39.0,0.0,0.0,0.0,0.0,0.0,38.0,79.0,64.0,NaN,81.013,0.0,NaN,0.000,0.0,NaN,0.000,Jackson,aljackson,Jackson,Alabama,01071,AL,Jackson
3,2023-10-29,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,0.0,0.0,90.0,76.0,0.0,0.0,0.0,0.0,0.0,77.0,101.0,77.0,NaN,76.238,70.0,NaN,90.909,8.0,NaN,7.921,Jefferson,aljefferson,Jefferson,Alabama,01073,AL,Jefferson
4,2023-10-29,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,0.0,0.0,103.0,81.0,0.0,0.0,0.0,0.0,0.0,83.0,155.0,129.0,NaN,83.226,62.0,NaN,74.699,0.0,NaN,0.000,Jefferson,aljefferson,Jefferson,Alabama,01073,AL,Jefferson
5,2023-10-29,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,0.0,0.0,0.0,149.0,130.0,0.0,0.0,0.0,0.0,0.0,144.0,249.0,104.0,NaN,41.767,0.0,NaN,0.000,0.0,NaN,0.000,Limestone,allimestone,Limestone,Alabama,01083,AL,Limestone
6,2023-10-29,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,0.0,1.0,0.0,124.0,98.0,0.0,0.0,0.0,0.0,0.0,100.0,82.0,77.0,NaN,93.902,0.0,NaN,0.000,5.0,NaN,6.098,Elmore,alelmore,Elmore,Alabama,01051,AL,Elmore
7,2023-10-29,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,0.0,0.0,0.0,201.0,89.0,0.0,0.0,0.0,0.0,0.0,89.0,139.0,111.0,NaN,79.856,0.0,NaN,0.000,0.0,NaN,0.000,Chilton,alchilton,Chilton,Alabama,01021,AL,Chilton
8,2023-10-29,015024,"LIMESTONE NURSING AND REHABILITATION CENTER, LLC",1600 WEST

In [42]:
nh_pre_proc_fips_merge.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,county_rev,state_county,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev
104363,2023-12-10,745017,THE LODGE OF SAGINAW HEALTH AND WELLNESS,848 W MCLEROY BLVD,SAGINAW,TX,76179,6859001210,Tarrant,Y,Y,0.0,4.0,0.0,130.0,101.0,0.0,0.0,2.0,0.0,0.0,110.0,144.0,91.0,63.194,63.194,0.0,0.0,0.0,0.0,0.0,0.0,Tarrant,txtarrant,Tarrant,Texas,48439,TX,Tarrant
104364,2023-12-10,745019,JAMES L. WEST ALZHEIMER'S CENTER,1111 SUMMIT AVE,FORT WORTH,TX,76102,8178771199,Tarrant,N,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant,txtarrant,Tarrant,Texas,48439,TX,Tarrant
104365,2023-12-10,745021,LINDALE SPECIALTY CARE CENTER,13905 FM 2710,LINDALE,TX,75771,4302602300,Smith,Y,Y,0.0,0.0,0.0,122.0,87.0,0.0,0.0,0.0,0.0,0.0,87.0,140.0,50.0,35.714,35.714,0.0,0.0,0.0,0.0,0.0,0.0,Smith,txsmith,Smith,Texas,48423,TX,Smith
104366,2023-12-10,745022,WARE MEMORIAL CARE CENTER,1510 S. VAN BUREN ST.,AMARILLO,TX,79101,8063730471,Potter,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Potter,txpotter,Potter,Texas,48375,TX,Potter
104367,2023-12-10,745038,TIERRA ESTE NURSING AND REHABILITATION CENTER,14300 PEBBLE HILLS BLVD,EL PASO,TX,79938,9159559998,El Paso,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,El Paso,txel paso,El Paso,Texas,48141,TX,El Paso


In [43]:
# Is the row count of nh_pre_proc_fips_merge the same as nh_pre_proc_df?
if len(nh_pre_proc_fips_merge.index) == len(nh_pre_proc_df.index):
    print(len(nh_pre_proc_fips_merge.index))
else:
    print(f'nh_pre_proc_fips_merge and nh_pre_proc_df do not have equal row counts. \n nh_pre_proc_fips_merge: {len(nh_pre_proc_fips_merge.index)},\n nh_pre_proc_df: {len(nh_pre_proc_df.index)}')
    

104368


In [44]:
# Convert week_ending to datetime
nh_pre_proc_fips_merge['week_ending'] = pd.to_datetime(nh_pre_proc_fips_merge['week_ending'])

In [45]:
# nh_pre_proc_fips_merge.loc[(nh_pre_proc_fips_merge['week_ending'] == nh_pre_proc_fips_merge['week_ending'].max()) 
#                    & (nh_pre_proc_fips_merge['federal_provider_number'] == '015010')]

In [46]:
# new_cols = ['CountyName','CountyFIPS', 'StateAbbr', 'STATE_COUNTY']

# How many fips codes are NULL?
null_rows = nh_pre_proc_fips_merge.loc[nh_pre_proc_fips_merge['CountyFIPS'].isnull()]
null_rows

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,county_rev,state_county,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev
226,2023-10-29,025015,WRANGELL MEDICAL CENTER LTC,232 WOOD STREET,WRANGELL,AK,99929,9078747000,Wrangell-Petersburg,Y,Y,0.0,0.0,0.0,18.0,12.0,0.0,0.0,0.0,0.00,0.00,12.0,46.0,42.0,NaN,91.304,0.0,NaN,0.000,0.0,NaN,0.000,Wrangell-Petersburg,akwrangell-petersburg,NaN,NaN,NaN,NaN,NaN
228,2023-10-29,025019,PETERSBURG MEDICAL CENTER LTC,103 FRAM STREET,PETERSBURG,AK,99833,9077724291,Wrangell-Petersburg,Y,Y,0.0,0.0,0.0,15.0,15.0,0.0,0.0,0.0,0.00,0.00,15.0,96.0,94.0,NaN,97.917,9.0,NaN,60.000,2.0,NaN,2.083,Wrangell-Petersburg,akwrangell-petersburg,NaN,NaN,NaN,NaN,NaN
231,2023-10-29,025024,PROVIDENCE SEWARD MOUNTAIN HAVEN,2203 OAK STREET,SEWARD,AK,99664,9072245241,Seward,Y,Y,4.0,2.0,0.0,40.0,38.0,0.0,0.0,2.0,105.26,0.00,39.0,107.0,99.0,NaN,92.523,30.0,NaN,76.923,0.0,NaN,0.000,Seward,akseward,NaN,NaN,NaN,NaN,NaN
235,2023-10-29,025028,CORDOVA COMMUNITY MED LTC,602 CHASE AVE,CORDOVA,AK,99574,9074248000,Cordova-Mccarthy,Y,Y,0.0,0.0,0.0,10.0,9.0,0.0,0.0,0.0,0.00,0.00,9.0,27.0,27.0,NaN,100.000,3.0,NaN,33.333,0.0,NaN,0.000,Cordova-Mccarthy,akcordova-mccarthy,NaN,NaN,NaN,NaN,NaN
239,2023-10-29,025034,PROVIDENCE VALDEZ MEDICAL CENTER,911 MEALS AVENUE,VALDEZ,AK,99686,9078341825,Valdez-Chitina-Whittier,Y,Y,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.00,0.00,10.0,64.0,47.0,NaN,73.438,7.0,NaN,70.000,0.0,NaN,0.000,Valdez-Chitina-Whittier,akvaldez-chitina-whittier,NaN,NaN,NaN,NaN,NaN
240,2023-10-29,025035,UTUQQANAAT INAAT,436 MISSION STREET,KOTZEBUE,AK,99752,9074423321,Not Available,Y,Y,0.0,0.0,0.0,18.0,17.0,0.0,0.0,0.0,0.00,0.00,17.0,47.0,47.0,NaN,100.000,15.0,NaN,88.235,28.0,NaN,59.574,Not Available,aknot available,NaN,NaN,NaN,NaN,NaN
243,2023-10-29,025038,MAPLE SPRINGS OF WASILLA,3265 E MERIDIAN LOOP,WASILLA,AK,99654,9078411217,Matanuska-Susitna,Y,Y,0.0,0.0,0.0,67.0,44.0,0.0,0.0,1.0,0.00,0.00,45.0,140.0,94.0,NaN,67.143,0.0,NaN,0.000,0.0,NaN,0.000,Matanuska-Susitna,akmatanuska-susitna,NaN,NaN,NaN,NaN,NaN
244,2023-10-29,025039,MAPLE SPRINGS OF PALMER,12130 EAST MAPLE SPRINGS WAY,PALMER,AK,99645,9078026641,Matanuska-Susitna,Y,Y,0.0,1.0,0.0,67.0,62.0,0.0,0.0,0.0,0.00,0.00,62.0,139.0,83.0,NaN,59.712,20.0,NaN,32.258,2.0,NaN,1.439,Matanuska-Susitna,akmatanuska-susitna,NaN,NaN,NaN,NaN,NaN
15138,2023-11-05,025015,WRANGELL MEDICAL CENTE

In [47]:
# sorted(list(null_rows.loc[null_rows['provider_state']=='AK','county_rev'].unique()),reverse=False)

# Are the fips null rows all for the state_county values that we cannot match on AK?
sorted(list(null_rows['state_county'].unique()),reverse=False)


['akcordova-mccarthy',
 'akmatanuska-susitna',
 'aknot available',
 'akseward',
 'akvaldez-chitina-whittier',
 'akwrangell-petersburg']

In [48]:
# What is the date range of week_ending?
max_date = max(nh_pre_proc_fips_merge['week_ending'])
max_date

Timestamp('2023-12-10 00:00:00')

In [49]:
min_date = min(nh_pre_proc_fips_merge['week_ending'])
min_date

Timestamp('2023-10-29 00:00:00')

In [50]:
# For each state_county, what are the nh's that passed qa and how many times in the date range?

In [51]:
# Send processed data to s3 or RWS WIP

# Split processed data into tables

In [52]:
# def find_location(row):
    
#     place = row['provider_name']
    
#     location = geolocator.geocode(place)
    
#     if location != None:
#         return location.address, location.latitude, location.longitude, location.raw['importance']
#     else:
#         return "Not Found", "Not Found", "Not Found", "Not Found"

In [53]:
# nh_pre_proc_df[['address', 'lat', 'lon', 'importance']] = nh_pre_proc_df.apply(find_location, axis="columns", result_type="expand")
# nh_pre_proc_df.head(10)